# Finding Synonyms and Analogies

This notebook is taken from a [PyTorch NLP tutorial](https://github.com/joosthub/pytorch-nlp-tutorial-ny2018/blob/master/day_1/0_Using_Pretrained_Embeddings.ipynb) source: [repository for the training tutorial as the 2018 O'Reilly AI Conference in NYC on April 29 and 30, 2018](https://github.com/joosthub/pytorch-nlp-tutorial-ny2018)

In [ ]:
## Colab SETUP
#!pip install annoy

In [ ]:
from annoy import AnnoyIndex
import numpy as np
import torch
from tqdm import notebook
import os
from pathlib import Path

Glove embeddings can be downloaded from [GloVe webpage](https://nlp.stanford.edu/projects/glove/).

You need to uncomment the appropriate part in the following cell

In [ ]:
## Colab SETUP
#!mkdir data
#%cd data
#!wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
#!unzip glove.6B.zip
#ROOT_DIR = 'content'
## local SETUP download glove in ~/data/ with the commands wget http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
## and unzip glove.6B.zip
#ROOT_DIR = Path.home()

In [ ]:
data_path = os.path.join(ROOT_DIR,'data/')
file = 'glove.6B.100d.txt'
glove_filename=data_path+file

In [ ]:
def load_word_vectors(filename):
    word_to_index = {}
    word_vectors = []
    
    with open(filename) as fp:
        for line in notebook.tqdm(fp.readlines(), leave=False):
            line = line.split(" ")
            
            word = line[0]
            word_to_index[word] = len(word_to_index)
            
            vec = np.array([float(x) for x in line[1:]])
            word_vectors.append(vec)
            
    return word_to_index, word_vectors

In [ ]:
word_to_index, word_vectors = load_word_vectors(glove_filename)

In [ ]:
len(word_vectors)

In [ ]:
word_vectors[0].shape

In [ ]:
word_to_index['beautiful']

In [ ]:
class PreTrainedEmbeddings(object):
    def __init__(self, glove_filename):
        self.word_to_index, self.word_vectors = load_word_vectors(glove_filename)
        self.word_vector_size = len(self.word_vectors[0])
        
        self.index_to_word = {v: k for k, v in self.word_to_index.items()}
        self.index = AnnoyIndex(self.word_vector_size, metric='euclidean')
        print('Building Index')
        for _, i in notebook.tqdm(self.word_to_index.items(), leave=False):
            self.index.add_item(i, self.word_vectors[i])
        self.index.build(50)
        print('Finished!')
    
    def get_embedding(self, word):
        return self.word_vectors[self.word_to_index[word]]
    
    def closest(self, word, n=1):
        vector = self.get_embedding(word)
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def closest_v(self, vector, n=1):
        nn_indices = self.index.get_nns_by_vector(vector, n)
        return [self.index_to_word[neighbor] for neighbor in nn_indices]
    
    def sim(self, w1, w2):
        return np.dot(self.get_embedding(w1), self.get_embedding(w2))


In [ ]:
glove = PreTrainedEmbeddings(glove_filename)

In [ ]:
glove.closest('apple', n=5)

In [ ]:
glove.closest('chip', n=5)

In [ ]:
glove.closest('baby', n=5)

In [ ]:
glove.closest('beautiful', n=5)

In [ ]:
def SAT_analogy(w1, w2, w3):
    '''
    Solves problems of the type:
    w1 : w2 :: w3 : __
    '''
    closest_words = []
    try:
        w1v = glove.get_embedding(w1)
        w2v = glove.get_embedding(w2)
        w3v = glove.get_embedding(w3)
        w4v = w3v + (w2v - w1v)
        closest_words = glove.closest_v(w4v, n=5)
        closest_words = [w for w in closest_words if w not in [w1, w2, w3]]
    except:
        pass
    if len(closest_words) == 0:
        print(':-(')
    else:
        print('{} : {} :: {} : {}'.format(w1, w2, w3, closest_words[0]))

In [ ]:
SAT_analogy('man', 'he', 'woman')

In [ ]:
SAT_analogy('fly', 'plane', 'sail')

In [ ]:
SAT_analogy('beijing', 'china', 'tokyo')

In [ ]:
SAT_analogy('man', 'woman', 'son')

In [ ]:
SAT_analogy('man', 'doctor', 'woman')

In [ ]:
SAT_analogy('woman', 'leader', 'man')